# Model Evaluation

In [6]:
# Load packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load custom modules
from custom_functions import fetch_data, prepare_data, train_predict, pol2cart

In [7]:
# Set the style of display
plt.style.use("ggplot")
%matplotlib inline

## Distortion Maps

In [9]:
# Fetch all the data taken from a given camera
camera = "mx"
image_num = "."
image_nums_train = ["2"]
image_nums_test = ["3"]
df = fetch_data(camera, image_num)

# Create the training and test datasets
X_train, y_train = prepare_data(df, camera, image_nums=image_nums_train)
X_test, y_test = prepare_data(df, camera, image_nums=image_nums_test)

custom_functions.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfn.dropna(inplace=True)


## Model Evaluation

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# build a Random Forest model
clf_RF = RandomForestRegressor(n_estimators=10, random_state=16)

# collect results on the learners
samples_100 = len(X_train)
results = {}
clf_name = clf_RF.__class__.__name__
clf = MultiOutputRegressor(clf_RF)
results[clf_name] = train_predict(clf_RF, samples_100, X_train, y_train, X_test, y_test)

RandomForestRegressor trained on 42 samples.
